# Initialisation de Spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Exemples avec les formats de stockage") \
    .getOrCreate()
sc = spark.sparkContext

25/11/04 11:35:10 WARN Utils: Your hostname, MacBook-Air-de-Theo.local resolves to a loopback address: 127.0.0.1; using 192.0.0.2 instead (on interface en0)
25/11/04 11:35:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/04 11:35:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/04 11:35:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/04 11:35:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/11/04 11:35:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/11/04 11:35:13 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


# Chargement des données
1. Créer un dataframe pour les prénoms

In [ ]:
prenoms = spark.read.load("prenomsParDeptsEtAnnees.gzip.parquet")
prenoms.createOrReplaceTempView("prenoms")
prenoms.show()

+----+----------+--------+---+-----+
|sexe|    prenom|effectif|dep|annee|
+----+----------+--------+---+-----+
|   2|    CARMEN|       3|972| 1934|
|   2|DONATIENNE|       3|972| 1934|
|   2|   MEGHANN|       3| 14| 1988|
|   2|  SÉVERINE|       3| 14| 1988|
|   2| MICHELINE|      40| 58| 1930|
|   2|    ODETTE|      40| 58| 1930|
|   1|    AMAURY|       9|  6| 2011|
|   1| GUILLAUME|       9|  6| 2011|
|   1|  ABDELLAH|       4| 59| 2012|
|   1|   MATISSE|       4| 59| 2012|
|   1|      ADEL|       3| 71| 2021|
|   1|     MILAN|       5| 71| 2021|
|   1|    QASSIM|       3| 71| 2021|
|   2|  GILBERTE|       3| 23| 1937|
|   2|    LOUISE|       3| 23| 1937|
|   1|    CHARLY|      14| 69| 2010|
|   1|      ERIC|       5| 69| 2010|
|   1|      SVEN|       5| 69| 2010|
|   2|      GABY|       3| 13| 2022|
|   2|     IRÈNE|       3| 13| 2022|
+----+----------+--------+---+-----+
only showing top 20 rows



1. Faire de même pour les départements

In [4]:
departements = spark.read.load("depts.parquet")
departements.createOrReplaceTempView("departements")
departements.show()

+---+---+--------+----+--------------------+--------------------+--------------------+
|dep|reg|cheflieu|tncc|                 ncc|              nccenr|             libelle|
+---+---+--------+----+--------------------+--------------------+--------------------+
| 52| 44|   52121|   3|         HAUTE MARNE|         Haute-Marne|         Haute-Marne|
| 53| 52|   53130|   3|             MAYENNE|             Mayenne|             Mayenne|
| 54| 44|   54395|   0|  MEURTHE ET MOSELLE|  Meurthe-et-Moselle|  Meurthe-et-Moselle|
| 55| 44|   55029|   3|               MEUSE|               Meuse|               Meuse|
| 56| 53|   56260|   2|            MORBIHAN|            Morbihan|            Morbihan|
| 57| 44|   57463|   3|             MOSELLE|             Moselle|             Moselle|
| 58| 27|   58194|   3|              NIEVRE|              Nièvre|              Nièvre|
| 59| 32|   59350|   2|                NORD|                Nord|                Nord|
| 60| 32|   60057|   5|                OISE

# Interroger les données

* La documentation sur Spark SQL est disponible [ici](https://spark.apache.org/docs/latest/sql-programming-guide.html) (de même pour [DataFrame](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.html))
* Pour chaque question, vous donnerez la réponse avec le DSL et en SQL

1. Donner, pour chaque prénom, son nombre d'occurences

## DSL

In [5]:
prenoms.groupBy("prenom").sum("effectif").show()

+--------------+-------------+
|        prenom|sum(effectif)|
+--------------+-------------+
|        ELIANE|          152|
|         AGNÈS|          216|
|       TATIANA|           42|
|     MICHELINE|           53|
|       AUGUSTE|          119|
|     ANASTASIA|           14|
|         HENRI|          119|
|_PRENOMS_RARES|         2688|
|         HAKAN|            3|
|          ELSA|           57|
|      SÉVERINE|           28|
|         JULIE|           72|
|         ANGEL|            9|
|          ALIX|           30|
|    CHRISTIANE|          310|
|      GILBERTE|           56|
|         CHRIS|            7|
|     GUILLAUME|          337|
|          ERIC|         1248|
|         ARTUR|            3|
+--------------+-------------+
only showing top 20 rows



## SQL

In [6]:
spark.sql("""SELECT p.prenom, SUM(p.effectif)
             FROM prenoms p
             GROUP BY p.prenom""").show()

+--------------+-------------+
|        prenom|sum(effectif)|
+--------------+-------------+
|        ELIANE|          152|
|         AGNÈS|          216|
|       TATIANA|           42|
|     MICHELINE|           53|
|       AUGUSTE|          119|
|     ANASTASIA|           14|
|         HENRI|          119|
|_PRENOMS_RARES|         2688|
|         HAKAN|            3|
|          ELSA|           57|
|      SÉVERINE|           28|
|         JULIE|           72|
|         ANGEL|            9|
|          ALIX|           30|
|    CHRISTIANE|          310|
|      GILBERTE|           56|
|         CHRIS|            7|
|     GUILLAUME|          337|
|          ERIC|         1248|
|         ARTUR|            3|
+--------------+-------------+
only showing top 20 rows



1. Donner le nombre total de naissances avec un prénom féminin

## DSL

In [14]:
prenoms.where("sexe = '2'").groupBy("sexe").sum("effectif").select("sum(effectif)").show()

+-------------+
|sum(effectif)|
+-------------+
|        39337|
+-------------+



## SQL

In [15]:
spark.sql("""SELECT SUM(effectif)
           FROM prenoms
           WHERE sexe = '2'""").show()

+-------------+
|sum(effectif)|
+-------------+
|        39337|
+-------------+



1. Donner l'effectif maximal et minimal par prénom

## DSL

In [23]:
from pyspark.sql.functions import min as _min, max as _max
prenoms.groupBy("prenom").agg(_min("effectif"),_max("effectif")).show()

+--------------+-------------+-------------+
|        prenom|min(effectif)|max(effectif)|
+--------------+-------------+-------------+
|        ELIANE|           11|           64|
|         AGNÈS|            3|           56|
|       TATIANA|            4|           19|
|     MICHELINE|            3|           40|
|       AUGUSTE|            3|           57|
|     ANASTASIA|            3|            7|
|         HENRI|            6|           68|
|_PRENOMS_RARES|            4|          582|
|         HAKAN|            3|            3|
|          ELSA|            9|           35|
|      SÉVERINE|            3|           19|
|         JULIE|            3|           36|
|         ANGEL|            3|            6|
|          ALIX|            3|           13|
|    CHRISTIANE|            6|           77|
|      GILBERTE|            3|           19|
|         CHRIS|            3|            4|
|     GUILLAUME|            4|           98|
|          ERIC|            3|          668|
|         

## SQL

In [19]:
spark.sql("""SELECT p.prenom, MIN(effectif), MAX(effectif)
           FROM prenoms p
           GROUP BY p.prenom""").show()

+--------------+-------------+-------------+
|        prenom|min(effectif)|max(effectif)|
+--------------+-------------+-------------+
|        ELIANE|           11|           64|
|         AGNÈS|            3|           56|
|       TATIANA|            4|           19|
|     MICHELINE|            3|           40|
|       AUGUSTE|            3|           57|
|     ANASTASIA|            3|            7|
|         HENRI|            6|           68|
|_PRENOMS_RARES|            4|          582|
|         HAKAN|            3|            3|
|          ELSA|            9|           35|
|      SÉVERINE|            3|           19|
|         JULIE|            3|           36|
|         ANGEL|            3|            6|
|          ALIX|            3|           13|
|    CHRISTIANE|            6|           77|
|      GILBERTE|            3|           19|
|         CHRIS|            3|            4|
|     GUILLAUME|            4|           98|
|          ERIC|            3|          668|
|         

1. Donner, pour chaque nom de département, le nombre moyen de prénoms depuis l'année 2000

## DSL

In [30]:
from pyspark.sql.functions import avg

prenoms.join(departements, prenoms.dep == departements.dep).filter("annee >= 2000").groupBy("ncc").avg("effectif").show()

+-----------------+------------------+
|              ncc|     avg(effectif)|
+-----------------+------------------+
|   SEINE ET MARNE|33.758620689655174|
|             NORD|15.606060606060606|
|            RHONE|11.657894736842104|
|            PARIS|13.676470588235293|
|       LA REUNION|              11.4|
|          MOSELLE|24.454545454545453|
|         YVELINES|13.515151515151516|
|  ALPES MARITIMES|11.041666666666666|
| BOUCHES DU RHONE|25.027027027027028|
|       MARTINIQUE|              69.0|
|SEINE SAINT DENIS| 6.944444444444445|
|   SEINE MARITIME|              13.5|
| LOIRE ATLANTIQUE|11.653846153846153|
|   SAONE ET LOIRE| 5.916666666666667|
|          AVEYRON| 5.666666666666667|
|    PAS DE CALAIS|22.458333333333332|
|         BAS RHIN|25.956521739130434|
|     VAL DE MARNE|              8.44|
|       VAL D OISE|              10.5|
|              VAR|               8.5|
+-----------------+------------------+
only showing top 20 rows



## SQL

In [31]:
spark.sql("""SELECT d.ncc, AVG(p.effectif)
              FROM prenoms p, departements d
              WHERE p.dep = d.dep AND p.annee >= 2000
              GROUP BY d.ncc""").show()

+-----------------+------------------+
|              ncc|     avg(effectif)|
+-----------------+------------------+
|   SEINE ET MARNE|33.758620689655174|
|             NORD|15.606060606060606|
|            RHONE|11.657894736842104|
|            PARIS|13.676470588235293|
|       LA REUNION|              11.4|
|          MOSELLE|24.454545454545453|
|         YVELINES|13.515151515151516|
|  ALPES MARITIMES|11.041666666666666|
| BOUCHES DU RHONE|25.027027027027028|
|       MARTINIQUE|              69.0|
|SEINE SAINT DENIS| 6.944444444444445|
|   SEINE MARITIME|              13.5|
| LOIRE ATLANTIQUE|11.653846153846153|
|   SAONE ET LOIRE| 5.916666666666667|
|          AVEYRON| 5.666666666666667|
|    PAS DE CALAIS|22.458333333333332|
|         BAS RHIN|25.956521739130434|
|     VAL DE MARNE|              8.44|
|       VAL D OISE|              10.5|
|              VAR|               8.5|
+-----------------+------------------+
only showing top 20 rows

